In [1]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd

In [2]:
df1 = pd.read_csv('store.csv')

In [3]:
df1['date'] = pd.to_datetime(df1['date'])
df1['item'] = df1['item'].astype(str)

In [4]:
# How is the behaviour of the top 10 most sold itens during the period?
fig1 = px.line(df1, x="date", y='qty', color='item', title='TOP 10 MOST SOLD ITENS DURING THE PERIOD')
fig1.update_layout(font_size=10, paper_bgcolor='#e6e693', plot_bgcolor='#e6e693', hoverdistance=10, spikedistance=5)
fig1.show()
#O item 185 só foi vendido até o início do outono-inverno 2018, quando os itens 408, 519 começaram a ser vendidos. 

In [5]:
# How is the behaviour of the top 10 most profitable itens during the period?
fig2 = px.line(df1, x="date", y='total', color='item', title='TOP 10 MOST PROFITABLE ITENS DURING THE PERIOD')
fig2.update_layout(font_size=10, paper_bgcolor='#e6e693', plot_bgcolor='#e6e693', hoverdistance=10, spikedistance=-1)
fig2.show()

In [6]:
#Quais  os item mais vendidos e o mais lucrativos

periods = [('2017-03-15', '2017-09-21'), ('2017-09-22', '2018-03-21'), ('2018-03-22', '2018-09-21'), ('2018-09-22', '2019-03-30')]
period_names = ['Spring-Summer 2017', 'Autumn-Winter 2017', 'Spring-Summer 2018', 'Autumn-Winter 2018']

# Top 10 most sold by Season
results_qty = {}
for i, (period_start, period_end) in enumerate(periods):
    current_period = df1[(df1['date'] >= period_start) & (df1['date'] <= period_end)]
    top_10_item_qty = current_period.groupby(['item'])[['qty']].sum().reset_index().sort_values(by='qty', ascending=False).head(10)
    period_name = f"{period_names[i]}: {period_start} - {period_end}"
    results_qty[period_name] = top_10_item_qty

# Top 10 most profitable by Season
results_total = {}
for i, (period_start, period_end) in enumerate(periods):
    current_period = df1[(df1['date'] >= period_start) & (df1['date'] <= period_end)]
    top_10_item_total = current_period.groupby(['item'])[['total']].sum().reset_index().sort_values(by='total', ascending=False).head(10)
    period_name = f"{period_names[i]}: {period_start} - {period_end}"
    results_total[period_name] = top_10_item_total

# Criar subplot com barras para as quantidades e os lucros
fig = make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=('Spring-Summer 2017', 'Autumn-Winter 2017', 'Spring-Summer 2018', 'Autumn-Winter 2018'), 
    column_widths=[2, 2] )

color_most_sold = 'rgb(0, 102, 204)'  # Azul
color_most_profit = 'rgb(255, 69, 0)'  # Vermelho

# Adicionar barras para as quantidades vendidas
for i, (period_name, top_10_products_qty) in enumerate(results_qty.items(), 1):
    trace_qty = go.Bar(x=top_10_products_qty['item'], y=top_10_products_qty['qty'], name=period_name, marker_color=color_most_sold)
    row_qty = (i - 1) // 2 + 1
    col_qty = (i - 1) % 2 + 1
    fig.add_trace(trace_qty, row=row_qty, col=col_qty)

# Adicionar barras para os lucros
for i, (period_name, top_10_products_total) in enumerate(results_total.items(), 1):
    trace_total = go.Bar(x=top_10_products_total['item'], y=top_10_products_total['total'], name=period_name, marker_color=color_most_profit)
    row_total = (i - 1) // 2 + 1
    col_total = (i - 1) % 2 + 1
    fig.add_trace(trace_total, row=row_total, col=col_total)

# Atualizar layout
fig.update_layout(
    title_text="MOST SOLD AND MOST PROFITABLE ITEMS BY SEASON",
    font_size=12,
    paper_bgcolor='#e6e693',
    plot_bgcolor='#e6e693',
    hoverdistance=10,
    spikedistance=-1)

fig.show()

In [7]:
#TOTAL PROFIT COMPARISION (comparação do lucro total)

lucro_por_item = df1.groupby('item')['total'].sum().sort_values(ascending=False).to_dict()

a = lucro_por_item['408'] + lucro_por_item['519']
b = lucro_por_item['185']

if a >= b:
    print(f"Os itens 408 e 519 compensaram em lucros o item 185 em {a - b} euro")
else:
    None

Os itens 408 e 519 compensaram em lucros o item 185 em 2533996 euro


In [8]:
#TOTAL SELLINGS COMPARISION (comparação das venda total)

vendas_por_item = df1.groupby('item')['qty'].sum().sort_values(ascending=False).to_dict()

c = vendas_por_item['408'] + vendas_por_item['519']
d = vendas_por_item['185']

if c >= d:
    print(f"Os itens 408 e 519 compensaram em vendas o item 185 em {c - d} unidades")
else:
    print(f"Os itens 408 e 519 NÃO compensaram em vendas o item 185 em u{c - d} unidades")

Os itens 408 e 519 NÃO compensaram em vendas o item 185 em u-619573 unidades


In [9]:
#Correlação entre temperatura e media de vendas de itens:
media = df1.groupby(['date', 'Celsius'])['qty'].mean().reset_index()
media['Celsius'] = media['Celsius'] * 100
fig3 = px.line(media, x="date", y='qty', color_discrete_sequence=['blue'],)
fig3.add_trace(px.line(media, x="date", y='Celsius', color_discrete_sequence=['red']).data[0])
fig3.update_layout(
    title='Correlação entre Temperatura e Média de Vendas de Itens', 
    xaxis_title='Data', 
    yaxis_title='Valor',
    font_size=12,
    paper_bgcolor='#3f254f',
    plot_bgcolor='#3f254f',
    hoverdistance=10,
    spikedistance=-1)

In [10]:
#Calculo de correlação:
corr = round(media.corr().loc['Celsius', 'qty'], 2)

if -1 <= corr < -0.5:
    print(f'Correlação negativa forte: {corr}')
elif -0.5 <= corr < 0:
    print(f'Correlação negativa fraca: {corr}')
elif 0 <= corr < 0.4:
    print(f'Correlação positiva fraca: {corr}')
elif 0.4 <= corr <= 1:
    print(f'Correlação positiva forte: {corr}')
else:
    None

Correlação positiva forte: 0.42


In [11]:
#Adivinhar qual o item:

print('''Com base nas informações precedentes, podemos tentar adivinhar qual o nome do item mais vendido e qual o mais lucrativo.''')

print('''Vimos que:
\n 1- Nosso dataset continha 83 itens, porém somente vimos os 10 itens mais vendidos e mais lucrativos,
\n 2- Estes itens foram analisados num tempo de 744 dias, abrangendo 8 estações, 2 primavera-verão e 2 outono-inverno,
\n 3- Havia uma diferença entre itens mais vendidos e itens mais lucrativos, de modo que nem todo item mais vendido era o mais lucrativo,
\n 4- O item 185 só foi vendido até o início do outono-inverno 2018, quando os itens 408 e 519 começaram a ser vendidos, o que sugere
um item da mesma categoria e estação, mas reformulado,
\n 5- Há uma correlação positiva forte de 0.42 entre a média da quantidade de vendas de itens e a temperatura média na cidade,
\n 6- O item mais vendido foi o 185, porém o mais lucrativo o 186.
''')

itens_disponiveis = df1['cloth'].unique()
itens_sold = df1.groupby(['item', 'cloth'])['qty'].sum().sort_values(ascending=False).reset_index().loc[0, 'cloth']
itens_prof = df1.groupby(['item', 'cloth'])['total'].sum().sort_values(ascending=False).reset_index().loc[0, 'cloth']

print('\n MAIS VENDIDO ----------------------------------------------------------------') 
print('Escolha um item da lista para adivinhar qual o nome do item mais vendido: \n')
for item in itens_disponiveis:
    print(f'- {item}')

item_escolhido = input('Digite o nome do item mais vendido: ')

if item_escolhido in itens_disponiveis and item_escolhido == itens_sold:
    print('Você acertou! \U0001F389 \U0001F389 \U0001F389 \U0001F389 \U0001F389')
else:
    print('Você errou! \U0001F480 \U0001F480 \U0001F480 \U0001F480 \U0001F480')

print('\n MAIS LUCRATIVO --------------------------------------------------------------')
print('Escolha um item da lista para adivinhar qual o nome do item mais lucrativo: \n')

item_escolhido = input('Digite o nome do item mais lucrativo: ')

if item_escolhido in itens_disponiveis and item_escolhido == itens_prof:
    print('Você acertou! \U0001F389 \U0001F389 \U0001F389 \U0001F389 \U0001F389')
else:
    print('Você errou! \U0001F480 \U0001F480 \U0001F480 \U0001F480 \U0001F480')

Com base nas informações precedentes, podemos tentar adivinhar qual o nome do item mais vendido e qual o mais lucrativo.
Vimos que:

 1- Nosso dataset continha 83 itens, porém somente vimos os 10 itens mais vendidos e mais lucrativos,

 2- Estes itens foram analisados num tempo de 744 dias, abrangendo 8 estações, 2 primavera-verão e 2 outono-inverno,

 3- Havia uma diferença entre itens mais vendidos e itens mais lucrativos, de modo que nem todo item mais vendido era o mais lucrativo,

 4- O item 185 só foi vendido até o início do outono-inverno 2018, quando os itens 408 e 519 começaram a ser vendidos, o que sugere
um item da mesma categoria e estação, mas reformulado,

 5- Há uma correlação positiva forte de 0.42 entre a média da quantidade de vendas de itens e a temperatura média na cidade,

 6- O item mais vendido foi o 185, porém o mais lucrativo o 186.


 MAIS VENDIDO ----------------------------------------------------------------
Escolha um item da lista para adivinhar qual o no

In [ ]:
# L'importanza dell'abbigliamento più venduto rispetto a quello più redditizio può dipendere dagli obiettivi specifici e dalle strategie del negozio. Entrambe le metriche hanno i loro vantaggi e possono essere rilevanti in contesti diversi. Ecco alcune considerazioni:

# Abbigliamento più venduto:
# Volume di vendite: Indica la popolarità di un articolo, mostrando quali prodotti hanno una domanda più ampia.
# Rotazione Rapida del Magazzino: Gli articoli più venduti contribuiscono spesso a una rotazione più veloce del magazzino, il che può essere vantaggioso per mantenere le collezioni aggiornate e allineate alle tendenze della moda.
# Attrazione di Clienti: Gli articoli popolari possono attirare più clienti nel negozio, risultando in opportunità di vendita incrociata e fidelizzazione del cliente.
# Abbigliamento più redditizio:
# Margine di Profitto: Indica quanto il negozio guadagna per ogni vendita. Gli articoli più redditizi possono contribuire in modo significativo al reddito del negozio, anche se hanno un volume di vendite inferiore.
# Valore Medio del Carrello: Gli articoli più costosi possono aumentare il valore medio degli acquisti dei clienti, influenzando positivamente il reddito totale.
# Focalizzazione sulla Redditività: Una strategia incentrata sulla redditività può essere importante per garantire la sostenibilità finanziaria del negozio, specialmente se dipende da margini più alti per coprire i costi operativi.